In [9]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt






In [10]:
# Check if CUDA is available and print the status
if torch.cuda.is_available():
    print("CUDA is available, using GPU")
    device = torch.device("cuda")
else:
    print("CUDA not available, using CPU")
    device = torch.device("cpu")

CUDA is available, using GPU


In [11]:
# Transform for data preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset and Dataloaders
train_dataset = datasets.ImageFolder(root='/kaggle/input/ddos-attack/attackImagesPaper/Train', transform=transform)
test_dataset = datasets.ImageFolder(root='/kaggle/input/ddos-attack/attackImagesPaper/Test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f'Number of classes: {len(train_dataset.classes)}')
print(f'Class to index mapping: {train_dataset.class_to_idx}')

Number of classes: 12
Class to index mapping: {'10_Syn.csvImgs': 0, '11_TFTP.csvImgs_12k': 1, '12_UDPLag_t.csvImgs': 2, '1_DrDoS_DNS.csvImgs': 3, '2_DrDoS_LDAP.csvImgs': 4, '3_DrDoS_MSSQL.csvImgs': 5, '4_DrDoS_NetBIOS.csvImgs': 6, '5_DrDoS_NTP.csvImgs': 7, '6_DrDoS_SNMP.csvImgs': 8, '7_DrDoS_SSDP.csvImgs': 9, '8_DrDoS_UDP.csvImgs': 10, '9_Normal_12k': 11}


In [12]:

model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 12)
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5, verbose=True)

In [13]:


def train_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if i % 100 == 99:
            print(f'Batch {i+1}, Loss: {running_loss/100:.3f}, Accuracy: {100.*correct/total:.2f}%')
            running_loss = 0.0
    
    epoch_accuracy = 100. * correct / total
    return epoch_accuracy, running_loss / len(dataloader)





In [14]:
def evaluate(dataloader, model, loss_fn):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    test_loss = running_loss / len(dataloader)
    test_accuracy = 100. * correct / total
    return test_loss, test_accuracy

In [15]:
num_epochs = 5
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    train_accuracy, train_loss = train_epoch(train_loader, model, loss_fn, optimizer)
    test_loss, test_accuracy = evaluate(test_loader, model, loss_fn)
    
    print(f'Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')
    print('-' * 40)
    
    # Learning rate scheduling
    scheduler.step(test_loss)

print('Training completed.')

Epoch 1/5
Batch 100, Loss: 0.440, Accuracy: 83.00%
Batch 200, Loss: 0.303, Accuracy: 85.17%
Batch 300, Loss: 0.267, Accuracy: 86.29%
Batch 400, Loss: 0.242, Accuracy: 86.90%
Batch 500, Loss: 0.231, Accuracy: 87.30%
Batch 600, Loss: 0.228, Accuracy: 87.75%
Batch 700, Loss: 0.224, Accuracy: 88.13%
Batch 800, Loss: 0.204, Accuracy: 88.45%
Batch 900, Loss: 0.212, Accuracy: 88.66%
Batch 1000, Loss: 0.199, Accuracy: 88.86%
Batch 1100, Loss: 0.192, Accuracy: 89.09%
Batch 1200, Loss: 0.209, Accuracy: 89.18%
Batch 1300, Loss: 0.194, Accuracy: 89.31%
Batch 1400, Loss: 0.210, Accuracy: 89.30%
Batch 1500, Loss: 0.200, Accuracy: 89.35%
Batch 1600, Loss: 0.192, Accuracy: 89.44%
Batch 1700, Loss: 0.185, Accuracy: 89.55%
Batch 1800, Loss: 0.213, Accuracy: 89.58%
Batch 1900, Loss: 0.196, Accuracy: 89.65%
Batch 2000, Loss: 0.202, Accuracy: 89.67%
Batch 2100, Loss: 0.181, Accuracy: 89.75%
Batch 2200, Loss: 0.185, Accuracy: 89.82%
Batch 2300, Loss: 0.201, Accuracy: 89.81%
Batch 2400, Loss: 0.195, Accuracy